In [ ]:
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage,ToolMessage
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from dotenv import load_dotenv
from IPython.display import Image, display
from typing import Literal
import os

print("✅ All imports successful")

In [ ]:
@tool
def get_weather(city: str) -> str:
    """Returns simulated weather for a given city."""
    fake_weather = {
        "lagos": "Sunny, 30°C",
        "london": "Cloudy, 18°C",
        "new york": "Rainy, 22°C"
    }
    return fake_weather.get(city.lower(), "Weather data not available.")


@tool
def define_word(word: str) -> str:
    """Returns the definition of a word."""
    dictionary = {
        "ephemeral": "Lasting for a very short time.",
        "ubiquitous": "Present, appearing, or found everywhere.",
        "resilient": "Able to recover quickly from difficulties."
    }
    return dictionary.get(word.lower(), "Definition not found.")


@tool
def web_search(query: str) -> str:
    """Search the web using DuckDuckGo."""
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=3)
        return "\n".join(r["body"] for r in results)


In [ ]:
#system prompt that defines assistant behavior 

sys_msg= SystemMessage(
    content="You are a friendly assistant but answers user question.Be careful and concise."
)
def assistant(state:MessagesState)-> dict:
    """
    The assistant node - processes messages and generates response.
    """

    # combine system prompt with conversation history
    messages=[sys_msg] + state["messages"]

    # Get response from LLM
    response=llm.invoke(messages)

    # Return as state update
    return {"messages": [AIMessage(content=response.content)]}

print("✅ Assistant node defined")


In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
tools = [get_weather, define_word, web_search]

llm_with_tools = llm.bind_tools(tools)


In [ ]:
builder = StateGraph(AgentState)

builder.add_node("agent", agent_node)
builder.add_node("tool", tool_node)

builder.set_entry_point("agent")

builder.add_conditional_edges(
    "agent",
    should_use_tool,
    {
        "tool": "tool",
        END: END
    }
)

builder.add_edge("tool", "agent")

app = builder.compile()

print("[OK] LangGraph agent ready!")


In [ ]:
#create a memory checkpointer (stores in memory)
memory=MemorySaver()

#compile the graph with memory
agent=builder.compile(checkpointer=memory)

In [ ]:
def run_agent(query: str):
    result = app.invoke(
        {"messages": [HumanMessage(content=query)]}
    )
    return result["messages"][-1].content
